## This is a notebook that verify the communication of the pointing kernel with M1M3 (including the rotator)
It is part of a series of notebook that tests the communication between the pointing kernel and different components:
- the mount
- the rotator
- M1M3
- M2
- Cam hexapod
- M2 hexapod
We will leave ComCam out of this notebook

The goals of the exercice are 
- Check heartbeat from all components.
- Make sure we can put all components in ENABLE state and back to STANDBY.
- Check the EFD to ensure that the correct events/commands/telemetry is published

This will ensure that we are ready to check the slew when needed


Library loading

In [ ]:
#from lsst.ts.observatory.maintel import MTCS
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

import asyncio

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime


## Set up -> things that would go into the tcs script

In [ ]:
now = datetime.now()

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
mtptg = salobj.Remote(script.domain, "MTPtg")
rot = salobj.Remote(script.domain, "Rotator")
m1m3 = salobj.Remote(script.domain, "M1M3")
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await asyncio.gather(mtptg.start_task,
                     script.start_task,
                     rot.start_task,
                     m1m3.start_task)

In [ ]:
await mtptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await mtptg.evt_summaryState.aget(timeout=5)

In [ ]:
await mtptg.cmd_start.start(timeout=5)

Check the state of the system

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(m1m3, settingsToApply = "Default", salobj.State.ENABLED)

Raise the mirror M1M3

In [ ]:
await m1m3.cmd_raiseM1M3.set_start(raiseM1M3=True, timeout = 30)

Enter the location for the telescope for the pointing

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

## Start of a slew

In [ ]:
now = datetime.now()
print("Start to point the telescope", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(0, unit= u.deg) 

In [ ]:
#This will become a function with Az, El and rot as an input and decide to start tracking
target_name="TMA motion test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

print(curr_time_mtptg.tai.value)

cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_mtptg.tai, 
                location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
# Calculating the other parameters     
rot_pa = rot_tel
await rot.cmd_trackStart.start(timeout=30.)


In [ ]:
#The pointing component is commanding the mount directly
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)
print("System Ready")

## Check telemetry

In [ ]:
target = await mtm.evt_target.aget() 
print('Target', target.tel_Elevation.aget()
rotAngle = await rot.evt_target.aget()

In [ ]:
m1m3Angle = await m1m3.tel_inclinometerData.aget()
print("m1m3Angle", m1m3Angle.inclinometerAngle)

In [ ]:
m1m3Position = await m1m3.imsData.aget()
print("m1m3Position x, y and z", m1m3Position.xPosition, m1m3Position.yPosition, m1m3Position.zPosition)

## Stop the system: stop the pointing kernel, lower the mirror what else?

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
#send the telescope to Park. Does not exist yet

In [ ]:
await m1m3.cmd_lowerM1M3(timeout=10.)
await m1m3.cmd_shutdown(timeout=10.)